In [1]:
!pip install --quiet openai faiss-cpu pandas numpy tiktoken

In [2]:
import os
from getpass import getpass
from openai import OpenAI

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key를 입력하세요: ")

client = OpenAI()
EMBED_MODEL = "text-embedding-3-small"
CHAT_MODEL  = "gpt-4.1-mini"


OpenAI API Key를 입력하세요: ··········


In [3]:
import pandas as pd

DATA_URL = "https://raw.githubusercontent.com/luminati-io/Yahoo-Finance-dataset-sample/main/Yahoo%20finance%20dataset.csv"

df_raw = pd.read_csv(DATA_URL)
print(df_raw.shape)
df_raw.head()
print(df_raw.columns)


(1000, 33)
Index(['name', 'company_id', 'entity_type', 'summary', 'stock_ticker',
       'currency', 'earnings_date', 'exchange', 'closing_price',
       'previous_close', 'open', 'bid', 'ask', 'day_range', 'week_range',
       'volume', 'avg_volume', 'market_cap', 'beta', 'pe_ratio', 'eps',
       'dividend_yield', 'ex_dividend_date', 'target_est', 'url',
       'people_also_watch', 'similar', 'risk_score', 'risk_score_text',
       'risk_score_percentile', 'recommendation_rating',
       'analyst_price_target', 'company_profile_address'],
      dtype='object')


In [4]:
# 사용할 주요 컬럼 이름
CANDIDATE_COLS = {
    "name": ["name"],
    "summary": ["summary", "description"],
    "ticker": ["stock_ticker", "ticker"],
    "exchange": ["exchange"],
    "closing_price": ["closing_price", "close"],
    "prev_close": ["previous_close"],
    "open": ["open"],
    "day_range": ["day_range"],
    "week_range": ["week_range"],
    "volume": ["volume"],
    "avg_volume": ["avg_volume"],
    "market_cap": ["market_cap"],
    "beta": ["beta"],
    "pe_ratio": ["pe_ratio"],
    "eps": ["eps"],
    "dividend_yield": ["dividend_yield"],
    "target_est": ["target_est"],
}

def pick_existing_column(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

COLUMN_MAP = {k: pick_existing_column(df_raw, v) for k, v in CANDIDATE_COLS.items()}
COLUMN_MAP


{'name': 'name',
 'summary': 'summary',
 'ticker': 'stock_ticker',
 'exchange': 'exchange',
 'closing_price': 'closing_price',
 'prev_close': 'previous_close',
 'open': 'open',
 'day_range': 'day_range',
 'week_range': 'week_range',
 'volume': 'volume',
 'avg_volume': 'avg_volume',
 'market_cap': 'market_cap',
 'beta': 'beta',
 'pe_ratio': 'pe_ratio',
 'eps': 'eps',
 'dividend_yield': 'dividend_yield',
 'target_est': 'target_est'}

In [5]:
def build_stock_text(row, colmap):
    def g(key, prefix=""):
        col = colmap.get(key)
        if col is None or col not in row or pd.isna(row[col]):
            return ""
        val = row[col]
        return f"{prefix}{val}"

    lines = []
    lines.append(f"[종목명] {g('name')}")
    if g("ticker"):
        lines.append(f"[티커] {g('ticker')}")
    if g("exchange"):
        lines.append(f"[거래소] {g('exchange')}")

    if g("closing_price"):
        lines.append(f"[현재가/종가] {g('closing_price')} ({g('day_range', '일중 가격범위: ')})")
    if g("week_range"):
        lines.append(f"[52주 가격범위] {g('week_range')}")
    if g("market_cap"):
        lines.append(f"[시가총액] {g('market_cap')}")
    if g("volume"):
        lines.append(f"[거래량] {g('volume')} (평균: {g('avg_volume')})")
    if g("beta"):
        lines.append(f"[베타(변동성 지표)] {g('beta')}")
    if g("pe_ratio"):
        lines.append(f"[PER] {g('pe_ratio')}")
    if g("eps"):
        lines.append(f"[EPS] {g('eps')}")
    if g("dividend_yield"):
        lines.append(f"[배당수익률] {g('dividend_yield')}")
    if g("target_est"):
        lines.append(f"[애널리스트 목표가 추정치] {g('target_est')}")

    if g("summary"):
        lines.append("\n[요약/설명]\n" + g("summary"))

    return "\n".join([ln for ln in lines if ln.strip()])

# 실제로 사용할 df
df = df_raw.copy()
stock_texts = df.apply(lambda r: build_stock_text(r, COLUMN_MAP), axis=1).tolist()

len(stock_texts), stock_texts[0][:500]


(1000,
 '[종목명] Ålandsbanken Abp (ALBAV.HE)\n[티커] ALBAV.HE\n[거래소] up\n[현재가/종가] 34.2 (일중 가격범위: 34.20 - 34.80)\n[52주 가격범위] 30.30 - 39.60\n[시가총액] 522234016.0\n[거래량] 351 (평균: 513)\n[베타(변동성 지표)] 0.205\n[PER] 9.421488\n[EPS] 3.63\n[배당수익률] 2.40 (7.04%)\n\n[요약/설명]\nHelsinki - Delayed Quote  • EUR')

In [ ]:
import numpy as np
import faiss

def get_embeddings(texts, batch_size=64):
    vectors = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        resp = client.embeddings.create(
            model=EMBED_MODEL,
            input=batch,
        )
        vecs = [d.embedding for d in resp.data]
        vectors.extend(vecs)
    return np.array(vectors, dtype="float32")

emb_matrix = get_embeddings(stock_texts)
print("임베딩 shape:", emb_matrix.shape)

dim = emb_matrix.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(emb_matrix)

id2row = df.reset_index(drop=True)


In [ ]:
def search_stocks(query, top_k=5):
    """
    query: 사용자 프로필/질문을 합친 자연어 문장
    """
    q_emb = get_embeddings([query])
    D, I = index.search(q_emb, top_k)
    I = I[0]

    results = []
    for rank, idx in enumerate(I):
        row = id2row.iloc[idx]
        res = {
            "rank": rank + 1,
            "row_idx": int(idx),
            "name": row.get(COLUMN_MAP["name"], ""),
            "ticker": row.get(COLUMN_MAP["ticker"], "") if COLUMN_MAP["ticker"] else "",
            "exchange": row.get(COLUMN_MAP["exchange"], "") if COLUMN_MAP["exchange"] else "",
            "closing_price": row.get(COLUMN_MAP["closing_price"], "") if COLUMN_MAP["closing_price"] else "",
            "market_cap": row.get(COLUMN_MAP["market_cap"], "") if COLUMN_MAP["market_cap"] else "",
            "pe_ratio": row.get(COLUMN_MAP["pe_ratio"], "") if COLUMN_MAP["pe_ratio"] else "",
            "dividend_yield": row.get(COLUMN_MAP["dividend_yield"], "") if COLUMN_MAP["dividend_yield"] else "",
            "text": stock_texts[idx],
        }
        results.append(res)
    return results

# 간단 테스트
test_results = search_stocks("미국 대형 성장주, 기술주 위주로 장기 투자")
for r in test_results:
    print(r["rank"], r["name"], r["ticker"], "| 거래소:", r["exchange"])


In [8]:
def recommend_with_rag(user_profile: str, user_question: str, top_k: int = 5) -> str:
    """
    user_profile: 나이, 투자기간, 성향, 월 투자 가능 금액 등 자유 서술
    user_question: 구체적인 요구 (예: '장기 성장주 위주로 2~3개 추천')
    """
    query = f"사용자 프로필: {user_profile}\n질문: {user_question}"
    candidates = search_stocks(query, top_k=top_k)

    if not candidates:
        return "조건에 맞는 후보 종목을 찾지 못했습니다."

    # 컨텍스트 텍스트 구성
    ctx_blocks = []
    for c in candidates:
        header = f"### 후보 {c['rank']}번: {c['name']} ({c['ticker']})\n"
        ctx_blocks.append(header + c["text"] + "\n")
    context = "\n\n".join(ctx_blocks)

    system_prompt = """
너는 주식/ETF 같은 상장 종목을 설명해주는 투자 도우미다.
항상 한국어로 답변하고, 과장되지 않게 리스크를 분명히 언급해야 한다.

규칙:
- 아래 컨텍스트에 포함된 종목들 중에서만 2~3개를 골라 추천한다.
- 각 종목마다:
  - 어떤 특징(섹터, 사업 내용, 성장성/안정성 등)이 있는지
  - 왜 이 사용자의 성향/기간/목표에 맞는지
  - 어떤 리스크(변동성, 섹터 리스크, 개별 기업 리스크 등)가 있는지
  를 구체적으로 설명한다.
- 절대 '무조건 오른다', '수익 보장' 같은 표현을 쓰지 말 것.
- 답변 마지막 줄에는 반드시 아래 문장을 그대로 넣는다:
  '※ 본 답변은 투자 권유가 아니며, 최종 투자 판단과 책임은 투자자 본인에게 있습니다.'
"""

    user_prompt = f"""
[사용자 프로필]
{user_profile}

[사용자 질문]
{user_question}

[후보 종목 정보 (공개 Yahoo Finance 샘플 데이터 기반)]
{context}
"""

    resp = client.responses.create(
        model=CHAT_MODEL,
        input=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
    )

    return resp.output[0].content[0].text


In [ ]:
user_profile = """
나이 28세, 사회 초년생.
월 투자 가능 금액: 30만 원 정도.
투자 기간: 5년 이상 장기 투자.
투자 성향: 공격적 (기술주, 성장주 위주로 높은 수익을 노리지만, 너무 극단적인 리스크는 피하고 싶음).
"""

user_question = "내 상황에서 고려해볼 만한 미국 상장 성장주/ETF 2~3개만 골라서, 이유와 리스크를 같이 설명해줘."

answer = recommend_with_rag(user_profile, user_question, top_k=7)
print(answer)


In [10]:
profile_for_chat = """
나이 30세, 데이터 엔지니어.
월 투자 가능 금액 20~40만 원.
투자 경험 조금 있음.
원금 전부 날리는 건 싫지만, 은행 예금보다는 높은 수익을 원함.
투자 기간은 3~7년.
"""

print("종목/ETF 추천 챗봇입니다. 종료하려면 q 입력.\n")
while True:
    q = input("궁금한 점을 입력하세요: ")
    if q.strip().lower() in ["q", "quit", "exit"]:
        print("대화를 종료합니다.")
        break
    ans = recommend_with_rag(profile_for_chat, q, top_k=7)
    print("\n[추천/설명]\n")
    print(ans)
    print("\n" + "="*80 + "\n")


종목/ETF 추천 챗봇입니다. 종료하려면 q 입력.

궁금한 점을 입력하세요: q
대화를 종료합니다.
